In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from geo_dist_prep.data import GEONAMES_DB
from geo_dist_prep.schemas.geoname import GeoName

engine = create_engine(f"sqlite:///../{GEONAMES_DB}")

Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
import pandas as pd

from sqlalchemy.sql import and_, select
from sqlalchemy import func, bindparam, Integer, text, Float, union_all
import matplotlib.pyplot as plt

from geo_dist_prep.schemas.helpers import *

southport = 26700924
lon, lat = -3.004175, 53.647599


queries = []
for dist in (10, 20, 40, 80, 160, 320):
    query = (
        session.query(GeoName)
        .filter(GeoName.score >= dist/10, nearby(lat, lon, dist))
        .order_by(func.random())
        .limit(5)
    )

    q = query.statement.compile()
    df = pd.read_sql_query(q, session.bind)
    plt.scatter(df.lon, df.lat)


In [ ]:
query =union_all(
    session.query(GeoName).filter(GeoName.country_code == "uk"),
    session.query(GeoName).filter(GeoName.country_code == "ie")
)

q = query.compile()
df = pd.read_sql_query(q, session.bind)

df